In [24]:
def load_prompt_from_md(md_path: str) -> str:
    """从Markdown文件中加载提示词"""
    try:
        with open(md_path, 'r', encoding='utf-8') as f:
            content = f.read()
        return content.strip()
    except Exception as e:
        print(f"加载特定提示词失败：{e}")
        return "请对以下AI风险事件进行分类。"

In [25]:
import json
input_dict = {}
result = []
with open("incident_summary.json", "r", encoding="utf-8") as f:
    data=json.load(f)
    for i in data:
        input_dict[i["incident_id"]]=i["generated_summary"]
instruction = load_prompt_from_md("risk_taxonomy_prompt.md")
with open("incidents_classification_label.json","r", encoding="utf-8") as f:
    for obj in json.load(f):
        incident_id= obj["incident_id"]
        input = input_dict.get(int(incident_id),"")
        output={
            "entity": obj.get("entity", 3),
            "intent": obj.get("intent", 3),
            "time": obj.get("time", 3),
            "eu_ai_act_risk_level": obj.get("eu_ai_act_risk_level", 4),
            "domain_classification": obj.get("domain_classification", []),
        }
        train_item = {
            "instruction": instruction,
            "input": input,
            "output": json.dumps(output),
        }
        result.append(train_item)
with open("sft_train_data.json", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

In [26]:
import random

# Split the result list into training and testing sets with a 4:1 ratio
random.shuffle(result)
split_index = int(len(result) * 0.8)  # 80% for training, 20% for testing
train_set = result[:split_index]
test_set = result[split_index:]

# Save the training and testing sets to separate JSON files
with open("train_set.json", "w", encoding="utf-8") as train_file:
    json.dump(train_set, train_file, ensure_ascii=False, indent=4)

with open("test_set.json", "w", encoding="utf-8") as test_file:
    json.dump(test_set, test_file, ensure_ascii=False, indent=4)

In [27]:
with open("test_set.json", "r", encoding="utf-8") as f:
    test_data = json.load(f)
    result=[{"instruction":item["instruction"],"input":item["input"],"output":""}for item in test_data]
with open("test_set_no_output.json", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=4)